In [ ]:
!pip install -q fuzzywuzzy python-dotenv 

In [ ]:
!pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from ast import literal_eval
from dotenv import load_dotenv
from fuzzywuzzy import fuzz
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import numpy as np
import os
import pandas as pd
import pickle
import requests

# Environment variables
Get your credentials for the Google Spreadsheet API via: https://developers.google.com/sheets/api/quickstart/python

Some credentials are stored in a .env file.
https://github.com/theskumar/python-dotenv

In [ ]:
# Load variables in .env file.
load_dotenv()

# Google Spreadsheet API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID')
SPREADSHEET_RANGE = 'Beer Club official list!B1:D'
local_server_port = 8000

# Untappd API tokens
UNTAPPD_CLIENT_ID = os.getenv('UNTAPPD_CLIENT_ID')
UNTAPPD_CLIENT_SECRET = os.getenv('UNTAPPD_CLIENT_SECRET')

# Filenames for data storage
beer_list_filename = 'beer_list.csv'
beer_data_filename = 'beer_data.csv'

# Google Spreadsheet API

In [ ]:
# Create connection with Google Spreadsheat
creds = None

# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=local_server_port)
    
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

In [ ]:
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=SPREADSHEET_RANGE).execute()
values = result.get('values', [])

# Create Pandas dataframe from spreadsheet values, including headers
beer_pd = pd.DataFrame(values[1:], columns=values[0])

# Data preparation

In [ ]:
beer_pd['Date'] = beer_pd['Date'].replace(r'', None, regex=True).ffill()

In [ ]:
# Drop Bringer column for privacy reasons
beer_pd = beer_pd.drop(['Bringer'], axis=1)

# Drop rows with multiple bringers
beer_pd = beer_pd.dropna(subset=['Beer'])

In [ ]:
# Drop rows of home brews and other miscellaneous
no_brewery = ["Videoland Weizen", "Diko's Imperial Stout", "dedataclub.nl White and Blue Label", "Miscellaneous"]
beer_pd = beer_pd[~beer_pd['Beer'].isin(no_brewery)]
# Fix Gooische Bierbrouwerij Blond and Goud

In [ ]:
beer_pd['Date'] = pd.to_datetime(beer_pd['Date'], format='%d-%m-%Y')
beer_pd['Month'] = beer_pd.Date.dt.month

In [ ]:
# Split rows with multiple beers during a session
beer_pd = beer_pd.drop('Beer', axis=1).join(beer_pd['Beer'].str.split('\n', expand=True)\
                             .stack().reset_index(level=1, drop=True).rename('Beer')).reset_index(drop=True)

In [ ]:
beer_pd.to_csv(beer_list_filename, index=False)

In [ ]:
len(beer_pd)

# Untappd API
https://untappd.com/api/docs/v4

In [ ]:
search_untappd = 'https://api.untappd.com/v4/search/beer'

In [ ]:
def search_beer_uptappd(x):
    try:
        response = requests.get(search_untappd, params = {'client_id': UNTAPPD_CLIENT_ID,
                                            'client_secret': UNTAPPD_CLIENT_SECRET,
                                            'q': x}).json()
        res_beers = response['response']['beers']

        beer = res_beers['items'][0]['beer'] if res_beers['count'] > 0 else {}
        brewery = res_beers['items'][0]['brewery'] if res_beers['count'] > 0 else {}

        beer_brewery = beer.copy()
        beer_brewery.update(brewery)

        return beer_brewery
    except:
        return None

## Split data enrichment to API limit
_"The default limit for API access is 100 calls per hour per key."_

In [ ]:
if os.path.isfile(beer_data_filename):
    beer_untappd_pd = pd.read_csv(beer_data_filename)
    beer_untappd_pd = beer_untappd_pd[beer_untappd_pd.beer_name.notnull()]
    
    beer_process_pd = beer_pd[beer_pd.Beer.isin(set(beer_pd.Beer).difference(beer_untappd_pd.Beer))]

else:
    beer_process_pd = beer_pd.copy()

print("Beers to be processed through Untappd:", len(beer_process_pd))

In [ ]:
untappd_pd = beer_process_pd.copy()
untappd_pd = untappd_pd.drop(['Date', 'Month'], axis=1).drop_duplicates()
untappd_pd = untappd_pd[:100]

In [ ]:
untappd_pd['untappd'] = untappd_pd.Beer.apply(search_beer_uptappd)
untappd_pd = untappd_pd[untappd_pd.untappd != {}]
len(untappd_pd)

In [ ]:
untappd_pd = untappd_pd.join(untappd_pd.untappd.apply(pd.Series))
untappd_pd = untappd_pd.drop(['untappd'], axis=1)
untappd_pd = untappd_pd.join(untappd_pd.location.apply(pd.Series))
untappd_pd = untappd_pd.drop(['location', 0], axis=1)
untappd_pd = untappd_pd.join(untappd_pd.contact.apply(pd.Series))
untappd_pd = untappd_pd.drop(['contact', 0], axis=1)

In [ ]:
if os.path.isfile(beer_data_filename):
    untappd_stored_pd = pd.read_csv(beer_data_filename).drop(['Date', 'Month'], axis=1)
    untappd_stored_pd = untappd_stored_pd[untappd_stored_pd.beer_name.notnull()]
    
    untappd_concat_pd = pd.concat([untappd_stored_pd, untappd_pd], sort=True).drop_duplicates()
else:
    untappd_concat_pd = untappd_pd[untappd_pd.beer_name.notnull()]

beer_untappd_pd = beer_pd.merge(untappd_concat_pd, on='Beer', how='left')
beer_untappd_pd.to_csv(beer_data_filename, index=False)

# Quality check

## Untappd ratio

In [ ]:
beer_list_len = len(beer_untappd_pd)
beer_untappd_len = len(beer_untappd_pd[beer_untappd_pd.beer_name.notnull()])
untappd_ratio = round((beer_untappd_len / beer_list_len)*100, 1)
print(f"Untappd ratio: {beer_untappd_len}/{beer_list_len} ({untappd_ratio}%)")

In [ ]:
beer_untappd_pd[~beer_untappd_pd.beer_name.notnull()]

## Name match ratio

In [ ]:
beer_untappd_pd['match_ratio'] = beer_untappd_pd.apply(lambda x: fuzz.partial_ratio(x['Beer'], x['brewery_name'] + " " + x['beer_name']) 
                                       if len(str(x['brewery_name'])) > 3 else None, axis=1)
beer_untappd_pd['match_ratio'].describe()

In [ ]:
beer_untappd_pd[beer_untappd_pd['match_ratio'] < 50]